## Includes

In [1]:
import sys
import os
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config

config_info = {
    "canalyst_api_key":"",
    "s3_access_key_id": "",
    "s3_secret_key": "",
    "fred_key": "",
    "default_dir": "",
    "mds_host": "",
    "wp_host": ""
}

config = Config(config=config_info)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

A configuration file has been created for you in 
/Users/jedgore/git/sdk-python/examples/notebooks/keys.json.


# Example: Top KPI in a Sector
## Workflow:
<li>Set switches
<li>Create ModelSet for target list
<li>Find common names
<li>Create a model_frame of common names
<li>Regress against target vector (in this case - alpha)
<li>List coefficients

## Control Switches: 
### Change ticker list as needed etc

In [2]:
target_list = ['CMA US',
              'ZION US',
              'WAL US',
              'FITB US',
              'CFR US',
              'HBAN US',
              'KEY US',
              'MTB US']
index_ticker = "^GSPC"
alpha_days = "alpha_1_day" #or alpha_5_day or alpha_10_day
regression_window = 8

### Create a modelset of our stock list

In [3]:
target_modelset = cd.ModelSet(ticker_list = target_list, config=config,allow_nulls=True)

### Get common time series names among the ModelSet

In [4]:
feature_list = target_modelset.common_time_series_names()
feature_list = [item for item in feature_list if 'EPS' not in item]
feature_list = [item for item in feature_list if 'StockPrice' not in item]
feature_list = [item for item in feature_list if 'FX' not in item]

### Run regressions for each ticker

In [5]:
kpi_d = []

for ticker in target_list:
    yahoo_ticker = ticker.split(" ")[0]
    df_data = target_modelset.pe_dataset(ticker,yahoo_ticker,index_ticker)
    categories = list(target_modelset.model_frame(ticker=ticker,
                      period_duration_type = "fiscal_quarter", 
                      mrq=True)['category'].unique())
    like_list = ['Stats','Analysis','Metrics','Segmented Results']
    category_filter = []
    for category in categories:
        for like in like_list:
            if like in category:
                category_filter.append(category)

    category_filter = list(set(category_filter))

    df_kpi = cdr.regress_dataframe_time_series_groups(df_data = df_data,
                                         y_name=alpha_days,
                                         return_grouped=True,
                                         category = category_filter,
                                         n_periods=regression_window)
    df_kpi = df_kpi[['ticker','time_series_description','time_series_name','category','rsquared']].sort_values('rsquared',ascending = True)
    df_kpi = df_kpi.groupby('time_series_description').first().reset_index().sort_values('rsquared',ascending=True)
    kpi_d.append(df_kpi)


### Concatenate our list and rank

In [6]:
df_all = pd.concat(kpi_d)
df_all = df_all[df_all['time_series_name'].isin(feature_list)]
df_all['count']=1
df_list = df_all.groupby('time_series_name').aggregate(lambda tdf: tdf.unique().tolist()).reset_index()[['time_series_name','ticker']]
df_all = df_all.groupby(['time_series_name','time_series_description']).agg({'rsquared':'mean', 'count':'sum',}).reset_index()
df_all = pd.merge(df_all,df_list,how='inner',left_on='time_series_name',right_on='time_series_name')
df_all.sort_values(['count','rsquared'],ascending=False)

,time_series_name,time_series_description,rsquared,count,ticker
7,MO_MA_OPEX,"OPEX Margin, %",0.35,8,"[CMA US, ZION US, WAL US, FITB US, CFR US, HBAN US, KEY US, MTB US]"
2,MO_GA_NIIGrowth,"Net Interest Income Growth, %",0.22,6,"[ZION US, WAL US, FITB US, CFR US, HBAN US, MTB US]"
3,MO_MA_Compensation,"Compensation Margin, %",0.18,6,"[ZION US, WAL US, FITB US, CFR US, HBAN US, KEY US]"
0,MO_GA_AssetGrowth,"Total Earning Assets - Avg. Balance Growth, %",0.15,6,"[CMA US, ZION US, WAL US, FITB US, CFR US, MTB US]"
1,MO_GA_LiabilityGrowth,"Total Interest-bearing Liabilities - Avg. Balance Growth, %",0.17,4,"[WAL US, HBAN US, KEY US, MTB US]"
4,MO_MA_EfficiencyRatio,"Efficiency Ratio (adj. One-time Items), %",0.11,4,"[CMA US, FITB US, CFR US, HBAN US, KEY US, MTB US]"
5,MO_MA_EfficiencyRatio,"Efficiency Ratio (adj. One-time items), %",0.15,1,"[CMA US, FITB US, CFR US, HBAN US, KEY US, MTB US]"
6,MO_MA_EfficiencyRatio,"Efficiency Ratio, %",0.04,1,"[CMA US, FITB US, CFR US, HBAN US, KEY US, MTB US]"
